## Setup

### Import libraries

In [1]:
from benchmarks.experiments.wrapper_overhead.common.deployment_utility import WrapperOverheadDeploymentUtility
from benchmarks.experiments.wrapper_overhead.common.file_utility import FileUtility
from benchmarks.experiments.wrapper_overhead.common.extended_aws_remote_client import ExtendedAWSRemoteClient
from multi_x_serverless.deployment.common.deploy.models.resource import Resource


import os
import yaml

### Setup Common Resources

In [2]:

from unittest.mock import MagicMock
from multi_x_serverless.deployment.common.deploy.deployment_packager import DeploymentPackager


runtime = "python3.8"
region = 'us-east-2'
client: ExtendedAWSRemoteClient = ExtendedAWSRemoteClient(region)
file_utility: FileUtility = FileUtility()

deployment_utility: WrapperOverheadDeploymentUtility = WrapperOverheadDeploymentUtility(region)
config: MagicMock = MagicMock()
deployment_packager: DeploymentPackager = DeploymentPackager(config)

In [3]:
dna_vis_direct_calls_dir = './dna_visualization/external_database/boto3_only_direct_calls' # Boto3 only direct calls

config = file_utility.get_config(dna_vis_direct_calls_dir)

config

{'config.yml': './dna_visualization/external_database/boto3_only_direct_calls/config.yml'} {'visualize': './dna_visualization/external_database/boto3_only_direct_calls/visualize', 'get_input': './dna_visualization/external_database/boto3_only_direct_calls/get_input'}


{'type': 'boto3_direct',
 'workload_name': 'workflow-wo-dna_vis-ed-direct',
 'starting_function_name': 'wo-dna_vis-ed-direct-get_input',
 'functions': {'wo-dna_vis-ed-direct-visualize': {'function_name': 'wo-dna_vis-ed-direct-visualize',
   'iam_policy_name': 'wo-dna_vis-ed-direct-visualize-policy',
   'sns_topic_name': 'wo-dna_vis-ed-direct-visualize-sns_topic',
   'timeout': 300,
   'memory': 512,
   'handler': 'app.visualize',
   'data_directory_path': './dna_visualization/external_database/boto3_only_direct_calls/visualize',
   'iam_policies_content': '{\n  "Version": "2012-10-17",\n  "Statement": [\n    {\n      "Action": [\n        "logs:CreateLogGroup",\n        "logs:CreateLogStream",\n        "logs:PutLogEvents"\n      ],\n      "Resource": "arn:aws:logs:*:*:*",\n      "Effect": "Allow"\n    },\n    {\n      "Action": ["dynamodb:GetItem", "dynamodb:UpdateItem"],\n      "Resource": "arn:aws:dynamodb:*:*:*",\n      "Effect": "Allow"\n    },\n    {\n      "Action": ["s3:GetObject

In [4]:
dna_vis_direct_calls_dir = './dna_visualization/external_database/boto3_only_direct_calls' # Boto3 only direct calls

deployment_utility.deploy_experiment(dna_vis_direct_calls_dir)

{'config.yml': './dna_visualization/external_database/boto3_only_direct_calls/config.yml'} {'visualize': './dna_visualization/external_database/boto3_only_direct_calls/visualize', 'get_input': './dna_visualization/external_database/boto3_only_direct_calls/get_input'}


KeyError: 'experiment_type'

{'experiment_type': 'boto3_direct',
 'functions': {'wo-dna_vis-ed-direct-visualize': {'function_name': 'wo-dna_vis-ed-direct-visualize',
   'iam_policy_name': 'wo-dna_vis-ed-direct-visualize-policy',
   'sns_topic_name': 'wo-dna_vis-ed-direct-visualize-sns_topic',
   'timeout': 300,
   'memory': 512,
   'handler': 'app.visualize',
   'runtime': 'python3.8',
   'data_directory_path': './dna_visualization/external_database/boto3_only_direct_calls/visualize',
   'iam_policies_content': '{\n  "Version": "2012-10-17",\n  "Statement": [\n    {\n      "Action": [\n        "logs:CreateLogGroup",\n        "logs:CreateLogStream",\n        "logs:PutLogEvents"\n      ],\n      "Resource": "arn:aws:logs:*:*:*",\n      "Effect": "Allow"\n    },\n    {\n      "Action": ["dynamodb:GetItem", "dynamodb:UpdateItem"],\n      "Resource": "arn:aws:dynamodb:*:*:*",\n      "Effect": "Allow"\n    },\n    {\n      "Action": ["s3:GetObject", "s3:PutObject"],\n      "Resource": "arn:aws:s3:::multi-x-serverless-im

In [9]:

import json




dna_vis_direct_calls_dir = './dna_visualization/external_database/boto3_only_direct_calls' # Boto3 only direct calls

files_dict, directories_dict = file_utility.get_files_and_directories(dna_vis_direct_calls_dir)
# For the files list, find the files that end with config.yaml
# If the file is found, then read the file and print the content

if 'config.yml' in files_dict:
    config_content = file_utility.load_yaml_file(files_dict['config.yml'])
    experiment_type = config_content['type']
    print(f'Experiment type: {experiment_type}')

    # Step 1: Go through all the folders in the directory_path
    # Those are the lambda functions that need to be deployed
    for folder_path in directories_dict.values():
        inner_files_dict, inner_directories_dict = file_utility.get_files_and_directories(folder_path)

        require_sns = False
        # print(inner_files_dict)
        # Step 2: For each folder, access the config to get the configuration
        # (this may also determine if sns is used) and iam policies files
        config_content = {}
        iam_policies_content = {}
        if 'configs' in inner_directories_dict:
            config_files_dict, _ = file_utility.get_files_and_directories(inner_directories_dict['configs'])

            # Load the config file
            if 'config.yml' in config_files_dict:
                config_content = file_utility.load_yaml_file(config_files_dict['config.yml'])
                # print(config_content)
            else:
                raise ValueError('config.yml not found in the configs directory')
            
            # Load the iam policies file
            if 'iam_policy.json' in config_files_dict:
                iam_policies_content = file_utility.load_yaml_file(config_files_dict['iam_policy.json'])
                # print(iam_policies_content)
            else:
                raise ValueError('iam_policy.json not found in the configs directory')
        else:
            raise ValueError('configs directory not found')
        
        # Lets get the path to the app.py and requirements.txt
        app_file_path = None
        requirements_file_path = None
        if 'app.py' in inner_files_dict:
            app_file_path = inner_files_dict['app.py']
        else:
            raise ValueError('app.py not found in the directory')
        
        if 'requirements.txt' in inner_files_dict:
            requirements_file_path = inner_files_dict['requirements.txt']
        else:
            raise ValueError('requirements.txt not found in the directory')
        
        # print(f'App file path: {app_file_path}')
        # print(f'Requirements file path: {requirements_file_path}')

        # At this point we have both a valid config and iam policy file
        # We can now deploy the lambda function
        function_name = config_content['function_name']
        iam_policy_name = f'{function_name}-policy'

        # print(f'Function name: {function_name}')
        # print(f'IAM Policy name: {iam_policy_name}')
        # print(f'Image name: {image_name}')

        # # Check if function and iam policy exists
        # # If it does, delete the old function and iam policy
        # if client.resource_exists(Resource(function_name, "function")):
        #     print(f'Deleting old lambda function: {function_name}')
        #     client.remove_function(function_name)
        # if client.resource_exists(Resource(iam_policy_name, "iam_role")):
        #     print(f'Deleting old IAM policy: {iam_policy_name}')
        #     client.remove_role(iam_policy_name)

        # Create new lambda function
        timeout = config_content['configs']['timeout']
        memory = config_content['configs']['memory']
        handler = config_content['handler']
        runtime = "python3.8"
        app_file_path # -> path to the app.py
        requirements_file_path # -> path to the requirements.txt


        # First lets print all the variables
        # print(f'Function name: {function_name}')
        # print(f'IAM Policy name: {iam_policy_name}')
        # print(f'Handler: {handler}')
        # print(f'Runtime: {runtime}')
        # print(f'App file path: {app_file_path}')
        # print(f'Requirements file path: {requirements_file_path}')
        # print(f'Timeout: {timeout}')
        # print(f'Memory: {memory}')

        # policy_arn = client.create_role(iam_policy_name, json.dumps(iam_policies_content), lambda_trust_policy)
        # print(f'Policy ARN: {policy_arn}')
        # break 
        policy_arn = "arn:aws:iam::485595969306:role/wo-dna_vis-ed-direct-visualize-policy"

        # Experiment type: boto3_direct
        # Function name: wo-dna_vis-ed-direct-visualize
        # IAM Policy name: wo-dna_vis-ed-direct-visualize-policy
        # Handler: app.visualize
        # Runtime: python3.8
        # App file path: ./dna_visualization/external_database/boto3_only_direct_calls/visualize/app.py
        # Requirements file path: ./dna_visualization/external_database/boto3_only_direct_calls/visualize/requirement.txt
        # Timeout: 300
        # Memory: 512
        # Also have iam_policies_content (or path if needed)

        # put_role_policy(self, role_name: str, policy_name: str, policy_document: str)
        # create_role(self, role_name: str, policy: str, trust_policy: dict)
        
        # Get the parent directory of the app file
        # parent_directory = os.path.dirname(app_file_path)
        # # print(f'Parent directory: {parent_directory}')

        # config.workflow_name = function_name
        # config.workflow_version = "1.0.0"
        # zip_path = deployment_packager._create_deployment_package(parent_directory, "python3.8")
        # print(f'Zip path: {zip_path}')

        # with open(zip_path, 'rb') as f:
        #     zip_contents = f.read()
    
        # function_arn = client.create_function(
        #     function_name = function_name,
        #     role_identifier = policy_arn,
        #     zip_contents = zip_contents,
        #     runtime = runtime,
        #     handler =  handler,
        #     environment_variables = {},
        #     timeout = timeout,
        #     memory_size = memory
        # )

        # print(f'Timeout: {}')
        # print(f'Memory: {memory}')
        function_arn = client.get_lambda_function(function_name)['FunctionArn']
        # print(function_arn)
        sns_topic_name = f'{function_name}-sns_topic'

        # print(f'Function ARN: {function_arn}')
        sns_topic_arn = client.create_sns_topic(sns_topic_name)

        print(f'SNS Topic ARN: {sns_topic_arn}')

        protocol = "lambda"
        subscriptionArn = client.subscribe_sns_topic(sns_topic_arn, protocol, function_arn)
        print(f'Subscription ARN: {subscriptionArn}')

        break


        # # Create the new function and iam policy
        # print(f'Creating new lambda function: {function_name}')
        # client.create_function(function_name, config_content['handler'], config_content['runtime'], config_content['role'], config_content['zip_file'])
        # print(f'Creating new IAM policy: {iam_policy_name}')
        # client.create_role(iam_policy_name, iam_policies_content['policy_document'])




        # if require_sns:
        #     sns_topic_name = f'{function_name}-sns_topic'
        #     # sns_topic_arn = deployment_utility.create_sns_topic(sns_topic_name)
        #     # print(f'SNS Topic ARN: {sns_topic_arn}')
        #     # deployment_utility.subscribe_lambda_to_sns_topic(function_name, sns_topic_arn)
        

        # # Step 3: Check if the lambda function and sns topic exists
        # # If it does, delete the old lambda function and sns topic



        # # print(inner_files_dict, inner_directories_dict)


    # Step 4: Create a zip file of the contents of the code and the requirements
    
    # Step 5: Create the lambda function, with the zip file and the configuration with docker
    
    # Step 6: And sns topic and subscribe the lambda function to the sns topic (if it requires it)

    # Step 7: verify that the lambda function and sns topic are created and correctly subscribed if needed


else:
    raise ValueError('Invalid experiment type: config.yml not found')

Experiment type: boto3_direct
Zip path: ./dna_visualization/external_database/boto3_only_direct_calls/visualize/.multi-x-serverless/deployment-packages/267ccfcd05a141cb69c11c21e335a329f1e10af416dea5bf5e572b345e417872-python3.8-wo-dna_vis-ed-direct-visualize-1.0.0.zip
SNS Topic ARN: arn:aws:sns:us-east-2:485595969306:wo-dna_vis-ed-direct-visualize-sns_topic
Subscription ARN: arn:aws:sns:us-east-2:485595969306:wo-dna_vis-ed-direct-visualize-sns_topic:a876c935-0927-4974-9ba9-8631ae7d8079


In [14]:
dna_vis_direct_calls_dir = './dna_visualization/external_database/boto3_only_direct_calls' # Boto3 only direct calls

files_dict, directories_dict = file_utility.get_files_and_directories(dna_vis_direct_calls_dir)
# For the files list, find the files that end with config.yaml
# If the file is found, then read the file and print the content

if 'config.yml' in files_dict:
    config_content = file_utility.load_yaml_file(files_dict['config.yml'])
    experiment_type = config_content['type']
    print(f'Experiment type: {experiment_type}')
else:
    raise ValueError('Invalid experiment type: config.yml not found')

Experiment type: boto3_direct


In [7]:
test_dic = {'config.yml': './dna_visualization/external_database/boto3_only_direct_calls/config.yml'}

if "config.yml" in test_dic:
    print("config.yml found")

config.yml found


In [4]:
dna_vis_direct_calls_dir = "./dna_visualization/external_database/boto3_only_direct_calls" # Boto3 only direct calls
dna_vis_sns_calls_dir = "./dna_visualization/external_database/boto3_only_sns" # Boto3 only sns calls
dna_vis_aws_step_functions_dir = "./dna_visualization/external_database/aws_step_function" # AWS Step Functions
dna_vis_multi_x_dir = "./dna_visualization/external_database/multi_x" # Multix

files_list, directories_list = file_utility.get_files_and_directories(dna_vis_aws_step_functions_dir)

# Print files and directories
print("Files:")
for file in files_list.items():
    print(file)

print("\nDirectories:")
for directory in directories_list.items():
    print(directory)

Files:
('config.yml', './dna_visualization/external_database/aws_step_function/config.yml')
('state_machine.json', './dna_visualization/external_database/aws_step_function/state_machine.json')

Directories:
('visualize', './dna_visualization/external_database/aws_step_function/visualize')
('get_input', './dna_visualization/external_database/aws_step_function/get_input')


In [5]:
json_file_path = "./dna_visualization/external_database/aws_step_function/state_machine.json"
print(file_utility.load_json_file(json_file_path))

{'Comment': 'A workflow to visualize DNA sequences.', 'StartAt': 'GetInput', 'States': {'GetInput': {'Type': 'Task', 'Resource': 'wo-dna_vis-ed-step_function-get_input', 'Next': 'Visualize'}, 'Visualize': {'Type': 'Task', 'Resource': 'wo-dna_vis-ed-step_function-visualize', 'End': True}}}


In [6]:
# Specify the path to your YAML file
yaml_file_path = './dna_visualization/external_database/boto3_only_direct_calls/config.yml'

print(file_utility.load_yaml_file(yaml_file_path))

{'type': 'boto3-direct', 'starting_function_name': 'wo-dna_vis-ed-direct-get_input'}


## Deploy Associated AWS functions

### DNA Visualization